### 学習データの準備

「肉」「魚」「茶」など14語のいずれかを含む文を取得する

In [33]:
# import sys
# print(sys.path)
# sys.path.append('src')
# print(sys.path)
# !pip3 install pandas
import solrindexer as indexer
import sqlitedatastore as datastore
import pandas as pd

if __name__ == '__main__':
    datastore.connect()
    # columnsを設定しておく
    df = pd.DataFrame(columns=['#label', 'doc_id', 'sentence_id', 'text'])
    
    results = indexer.search_annotation(
        fl_keyword_pairs=[
            ('sentence_txt_ja', [['肉','魚','茶','塩','野菜','油','森林','砂漠','草原','海','木材','果樹','麦','米']]),
            ('name_s', [['sentence']]),
        ],
        rows=1000)
    
    for r in results['response']['docs']:
        # テキストの取得
        text = datastore.get(r['doc_id_i'], ['content'])['content']
        # センテンスの取得
        sent = datastore.get_annotation(r['doc_id_i'], 'sentence')[r['anno_id_i']]
        
        # Seriesで1行のdfを作成
        df_s = pd.Series( [0, r['doc_id_i'], r['anno_id_i'], text[sent['begin']:sent['end']]], index=df.columns )
        # dfにSeriesを挿入する
        df = df.append(df_s, ignore_index=True)
        
    datastore.close()
    display(df)

,#label,doc_id,sentence_id,text
0,0,124,204,主食の米に、油と塩を多用する
1,0,91,349,主食は米、魚、野菜、果物である
2,0,177,465,魚・肉の料理が比較的に少なく、スープや野菜の煮物などの料理が圧倒的に多い
3,0,120,165,肉、魚、野菜などの具材を炒めてからスープで煮込んだ後、同じスープで米を炊き込むベンヌチン（1...
4,0,90,122,主に、パーム油、コーヒー、木材
...,...,...,...,...
995,0,174,300,1948年のマーシャル・プランと強い文化的な結びつきが両国の絆を強めたが、シュレーダー首相の...
996,0,196,209,日本は資本主義体制をとり日米同盟を維持しているが、キューバに対しては地理的・政治的な利害関係...
997,0,19,112,またカリオン（英語版）で発見されたヒエログリフやンガリンジェリ族のラミンジェリ人の高齢者の主...
998,0,55,146,一方、豊臣秀吉が李氏朝鮮に侵攻した文禄・慶長の役の失敗後、1603年に徳川家康が開いた江戸幕...


### DataFlameをcsvで出力する

エクセルなどで、正解ラベルを自分で入力する

In [35]:
df.to_csv("svmdata.csv")

### 特徴量を入力したデータを確認

In [39]:
df = pd.read_csv('data/svmdata100.csv', header=0, index_col=0, )
df

,#label,doc_id,sentence_id,text
0,0,124,204,主食の米に、油と塩を多用する
1,0,91,349,主食は米、魚、野菜、果物である
2,1,177,465,魚・肉の料理が比較的に少なく、スープや野菜の煮物などの料理が圧倒的に多い
3,0,120,165,肉、魚、野菜などの具材を炒めてからスープで煮込んだ後、同じスープで米を炊き込むベンヌチン（1...
4,1,90,122,主に、パーム油、コーヒー、木材
...,...,...,...,...
96,1,50,163,また茶を飲む習慣も広まったが、茶は砂糖とミントを入れて沸騰させ濃いめにし小さなグラスで3回に...
97,1,93,110,国土の多くは草原となっており馬や牛や羊が飼育されている
98,0,168,245,西にアンデス山脈、東にはパンパと呼ばれる大草原が広がる
99,0,177,37,ウクライナの国土のほとんどは、肥沃な平原、ステップ（草原）、高原で占められている


### 学習

テキストから特徴量を抽出する関数と分類を実行する関数を作成、学習を行う関数も作成

In [111]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

# BoW形式の特徴量に変換する
# gensimのCountVectorizerをanalyzer='word'で実行する事で、半角スペースを単語区切りとみなしたBoWベクトルを作成できる
def vectorize(contents, vocab=None):
    vectorizer = CountVectorizer(analyzer='word', vocabulary=vocab)
    vecs = vectorizer.fit_transform(contents)
    vocab = vectorizer.vocabulary_
    return vecs, vocab

# テキストから特徴量を抽出する関数
def convert_into_features(sentences, vocab=None):
    contents = []
    for doc_id, sent, tokens in sentences:
        # 単語で分割
        lemmas = [token['lemma'] for token in tokens if token['POS'] in ['名詞', '動詞']]
        # 半角スペースで区切り
        content = ' '.join(lemmas)
        contents.append(content)
    # 入力されたテキストに含まれる単語をもとにしたボキャブラリを自動で作成している
    features, vocab = vectorize(contents, vocab=vocab)
    return features, vocab

# 学習の際も分類の際も同じvocabを使用する為
def convert_into_features_using_vocab(sentences, vocab):
    features, _ = convert_into_features(sentences, vocab)
    return features

# 学習
def train(labels, features):
    model = svm.LinearSVC()
    model.fit(features, labels)
    return model

# 分析結果を返す
def classify(features, model):
    predicts = model.predict(features)
    return predicts

### 実行

In [121]:
import time
from sklearn.externals import joblib
from annoutil import find_xs_in_y

if __name__ == '__main__':
    datastore.connect()
    # ラベル付きデータ読み込み
    sentences = []
    labels = []
    df = pd.read_csv('data/svmdata100.csv', header=0, index_col=0, )
    # dfをzipを使い複数列、1行ずつ取り出す
    for label, doc_id, sent_id in zip(df['#label'].astype('int64'), df['doc_id'], df['sentence_id'].astype('int64')):
        sent = datastore.get_annotation(doc_id, 'sentence')[sent_id]
        tokens = find_xs_in_y(datastore.get_annotation(doc_id, 'token'), sent)
        sentences.append((doc_id, sent, tokens))
        labels.append(label)
    
    # 学習データ特徴量生成
    # 冒頭の8割を学習用データとする
    num_train = int(len(sentences) * 0.8)
    sentences_train = sentences[:num_train]
    labels_train = labels[:num_train]
    # 特徴量の作成
    features, vocab = convert_into_features(sentences_train)

    # 学習
    time_s = time.time()
    print(':::TRAIN START')
    model = train(labels_train, features)
    print(':::TRAIN FINISHED', time.time() - time_s)
    
    # 学習モデルをファイルに保存
    joblib.dump(model, 'result/model.pkl')
    joblib.dump(vocab, 'result/vocab.pkl')
    
    # 分類の実行
    features_test = convert_into_features_using_vocab(sentences[num_train:], vocab)
    predicteds = classify(features_test, model)
    for predicted, (doc_id, sent, tokens), label in zip(predicteds, sentences[num_train:], labels[num_train:]):
        # 結果の確認
        text = datastore.get(doc_id, ['content'])['content']
        if predicted == label:
            print('correct ', ' ', label, predicted, text[sent['begin']:sent['end']])
        else:
            print('incorrect', ' ', label, predicted, text[sent['begin']:sent['end']])
    datastore.close()

:::TRAIN START
:::TRAIN FINISHED 0.0009210109710693359
correct    1 1 農林産品としてはこのほか、カシューナッツや木材の輸出もある
correct    0 0 ヨーロッパ最大の草原のひとつ （ホルトバージ国立公園）
correct    0 0 なお紅茶ではなく中国茶が多く飲まれている[32]
incorrect   0 1 フィリピン海、南シナ海、セレベス海に囲まれる
correct    1 1 オイルパーム（アブラヤシ）から精製されるパームオイル（パーム油、ヤシ油）は、植物油の原料の一つで、1990年代後半日本国内では菜種油・大豆油に次いで第3位で、食用・洗剤・シャンプー・化粧品の原料として需要の増大が見込まれている
correct    0 0 この砂漠は風によって絶えず景色が変化する砂砂漠（エルグ）、ごつごつした石や砂利と乾燥に強い植物がわずかにみられる礫砂漠、または植生のない岩肌がむき出しになっている岩石砂漠が広がっており、砂砂漠は一部であり、大部分は礫砂漠と岩石砂漠で占められている[10]
incorrect   0 1 国土を囲む海域には北極海の一部であるバレンツ海、白海、カラ海、ラプテフ海、東シベリア海と、太平洋の一部であるベーリング海、オホーツク海、日本海、そして西のバルト海と西南の黒海があり、海岸線は37,000kmに及ぶ
correct    1 1 羊肉はウズベキスタン国内でヒツジの放牧が盛んであることから一般的に販売されている肉であり、様々なウズベキスタン料理に使用されている
correct    0 0 大鑽井盆地より更に西はグレートサンディ砂漠、グレートビクトリア砂漠、ギブソン砂漠等の砂漠が広がり、人はあまり住んでいない
correct    1 1 資源には見るべきものがなく、わずかに塩が生産されるのみである
correct    1 1 塩を4万トン生産する
incorrect   0 1 ガンディーは「塩の行進」を開始したが成功しなかった
correct    1 1 鉱業の対象となる唯一の資源は塩である
correct    0 0 東はアドリア海、西でティレニア海とリグリア海、南でイオニア海と地中海に面している
correct    1 1 有機鉱物以外で

### 学習済みモデルの中身の確認を行う。

In [122]:
if __name__ == '__main__':
    model = joblib.load('result/model.pkl')
    vocab = joblib.load('result/vocab.pkl')
    
    # 学習済みモデルの確認
    # 各単語の重みを確認する、重みの絶対値の大きい語が、影響を与えた語となる
    for weight, word in sorted(zip(model.coef_[0], vocab), reverse=True):
        print(f'{weight:f} {word:s}')

0.550046 同様
0.550044 砂漠
0.423106 ジャガイモ
0.371718 栽培
0.291349 高原
0.291349 通じる
0.287340 輸出
0.283142 菓子
0.265910 つのる
0.257392 機関
0.252625 流域
0.248882 観光
0.248466 コーヒー
0.248159 細分
0.248159 ニンジン
0.238296 塩分
0.234492 その他
0.230789 石灰岩
0.225380 コプラ
0.216863 適地
0.216863 全国
0.214036 木材
0.209064 定める
0.207639 ラウ
0.207639 ガス
0.206501 衣料
0.206365 製品
0.206365 産品
0.206365 沿い
0.206365 コブ
0.204666 圧倒的
0.192058 豚肉
0.191571 一般
0.190049 雨林
0.190049 地形
0.190049 出す
0.182758 見る
0.176486 主食
0.176240 煮込む
0.159234 体制
0.157852 ほか
0.143632 多湿
0.143632 サラダ油
0.143632 アスパラガス
0.133497 基準
0.133497 公正
0.133224 様々
0.133224 復興
0.127506 チーズ
0.126438 青果物
0.126438 提供
0.126438 アンデス山脈
0.126438 もの
0.125872 牧場
0.124472 森林地帯
0.124472 ルート
0.124472 バナナ
0.124233 食べる
0.124233 出回る
0.124233 バラ
0.123081 利用
0.118258 香り
0.118258 食品
0.118258 チリ
0.110546 魚介
0.110546 例外
0.110546 イギリス
0.110546 つける
0.110259 大半
0.110259 北部
0.110259 きく
0.106148 植物
0.106148 国境
0.106148 団子
0.096029 経済
0.096029 中央
0.096029 ハム
0.095196 つくる
0.087491 進出
0.083240 畜産
0.0